In [31]:
with open("nom_artiste.txt", "r") as fichier:
    nom = fichier.read()
with open('KEY.txt','r') as f:
    api_key = f.read()

k=5
ls_nom = list(set(nom.strip('{}').replace("'", "").split(',')))
taille_sous_set = len(ls_nom) // k
reste = len(ls_nom) % k

# Découper en 5 sous-sets
sous_sets = []
start = 0

for i in range(5):
    end = start + taille_sous_set + (1 if i < reste else 0)  # Ajouter un élément en plus pour les premiers 'reste' sous-ensembles
    sous_sets.append(set(ls_nom[start:end]))
    start = end

liste_nom = []
for i in sous_sets:
    liste_nom.append(str(i))

In [60]:
import os
import google.generativeai as genai

genai.configure(api_key=api_key)

# Create the model
generation_config = {
  "temperature": 0.6,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        "Voici une liste de noms d'artistes, renvoi moi un dictionnaire contenant en clé le nom de l'artiste et en valeur sa nationalité : [inox,michou,gotaga]",
      ],
    },
    {
      "role": "model",
      "parts": [
        "```python\nartistes = {\n    \"inox\": \"Française\",\n    \"michou\": \"Française\",\n    \"gotaga\": \"Française\"\n}\n``` \n",
      ],
    },
  ]
)

liste_reponses = []
for txt in liste_nom:
  response = chat_session.send_message(txt)
  liste_reponses.append(response.text)

In [62]:
l_sauvegard = liste_reponses
liste_reponses

['```python\nartistes = {\n    \'Fleuves\': \'Française\',\n    \'Ash Code\': \'Française\',\n    \'Théa\': \'Française\',\n    \'Odyssey\': \'Française\',\n    \'Christophe Rousset\': \'Française\',\n    \'William\': \'Française\',\n    \'TURFU\': \'Française\',\n    \'The Allergies\': \'Britannique\',\n    \'JOhnny Montreuil\': \'Française\',\n    \'Kaelin Ellis\': \'Américaine\',\n    \'Geminiani\': \'Italienne\',\n    \'Maurizio Minardi Trio\': \'Italienne\',\n    \'Ośka\': \'Polonaise\',\n    \'Theobuntu\': \'Française\',\n    \'Sean Mason\': \'Américaine\',\n    \'Baba Zula\': \'Turque\',\n    \'Alison Wheeler\': \'Américaine\',\n    \'Yann Beuron\': \'Française\',\n    \'Stefan Pop\': \'Roumaine\',\n    \'Youn Sun Nah\': \'Coréenne\',\n    \'Deniz Mahir Kartal\': \'Turque\',\n    \'Rhizottome\': \'Française\',\n    \'Charlotte Wassy\': \'Française\',\n    \'Furyan\': \'Japonaise\',\n    \'Équipe de Foot\': \'Française\',\n    \'Rolland Merguez\': \'Française\',\n    \'TAP FACTOR

In [64]:
import re

for i in range(k):
    liste_reponses[i] = liste_reponses[i].replace('\n', '').replace(' ','')

for i in range(k):
    resultat = re.search(r'\{(.*)\}', liste_reponses[i])
    ret = resultat.group()
    liste_reponses[i] = ret
liste_reponses

['{\'Fleuves\':\'Française\',\'AshCode\':\'Française\',\'Théa\':\'Française\',\'Odyssey\':\'Française\',\'ChristopheRousset\':\'Française\',\'William\':\'Française\',\'TURFU\':\'Française\',\'TheAllergies\':\'Britannique\',\'JOhnnyMontreuil\':\'Française\',\'KaelinEllis\':\'Américaine\',\'Geminiani\':\'Italienne\',\'MaurizioMinardiTrio\':\'Italienne\',\'Ośka\':\'Polonaise\',\'Theobuntu\':\'Française\',\'SeanMason\':\'Américaine\',\'BabaZula\':\'Turque\',\'AlisonWheeler\':\'Américaine\',\'YannBeuron\':\'Française\',\'StefanPop\':\'Roumaine\',\'YounSunNah\':\'Coréenne\',\'DenizMahirKartal\':\'Turque\',\'Rhizottome\':\'Française\',\'CharlotteWassy\':\'Française\',\'Furyan\':\'Japonaise\',\'ÉquipedeFoot\':\'Française\',\'RollandMerguez\':\'Française\',\'TAPFACTORY\':\'Américaine\',\'Leho\':\'Estonienne\',\'Rocé\':\'Française\',\'Spitnoise\':\'Française\',\'SylvainRansy\':\'Française\',\'MarleneLarsen\':\'Danoise\',\'MADAM\':\'Française\',\'NawelBenKraiem\':\'Tunisienne\',\'OmarRudberg\':\'

In [68]:
import ast
dico_natio = {}
for i in range(k):
    my_dict = ast.literal_eval(liste_reponses[i])
    dico_natio = dico_natio|my_dict


In [73]:
type(dico_natio)

dict

In [74]:
import json

with open('artiste_natio.json', 'w', encoding='utf-8') as fichier:
    json.dump(dico_natio, fichier, ensure_ascii=False, indent=4)